In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import numpy as np
import os

dataset_folder = 'path/to/your/dataset_folder'  
img_size = (32, 32)  
batch_size = 32
num_classes = len(os.listdir(dataset_folder)) 

train_datagen = ImageDataGenerator(
    rescale=1./255,  
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2  
)

train_generator = train_datagen.flow_from_directory(
    dataset_folder,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse', 
    subset='training',   
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    dataset_folder,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation',
    shuffle=False
)

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator,
    verbose=1
)

test_loss, test_acc = model.evaluate(val_generator)
print(f"Test Accuracy: {test_acc:.4f}")

y_pred = []
y_true = []
for images, labels in val_generator:
    preds = np.argmax(model.predict(images), axis=1)
    y_pred.extend(preds)
    y_true.extend(labels)
    if len(y_pred) >= val_generator.samples:
        break

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

report = classification_report(y_true, y_pred, target_names=class_names)
print("Classification Report:\n", report)

class_names = list(train_generator.class_indices.keys())
print("Class Names:", class_names)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 492s 3us/step


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 30s 18ms/step - accuracy: 0.3498 - loss: 1.7863
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 33s 21ms/step - accuracy: 0.4985 - loss: 1.4111
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 33s 21ms/step - accuracy: 0.5381 - loss: 1.3055
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.5531 - loss: 1.2645
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 39s 25ms/step - accuracy: 0.5672 - loss: 1.2276
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6034 - loss: 1.1304
Test Accuracy: 0.6066
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Confusion Matrix:
 [[619  46  42  17  38  11  35  26  67  99]
 [ 14 739   4   9  12   2  29  15   8 168]
 [ 72   9 290  50 219  90 177  64   7  22]
 [ 10  10  43 328 120 189 202  70   4  24]
 [ 22   4  29  44 582  30 171 103  10   5]
 [  5   5  32 128  92 529  98  90   6  15]
 [  4   2  12  22  50  15 874   6   5  10]
 [  9   4  15  33  78  62  28 734   1  36]
 [ 93  90  12  22  23  15  21   8 610 106]
 [ 18 113   5  13   9